## Writing an Instant Messaging Server Side

In [ ]:
mkdir -p src/main/java/name/jhub/im/server

In [ ]:
cat << EOF > src/main/java/name/jhub/im/server/StartServer.java
package name.jhub.im.server;


import name.jhub.im.handler.ConnectionHandler;

import io.netty.bootstrap.ServerBootstrap;
import io.netty.channel.ChannelFuture;
import io.netty.channel.ChannelHandler;
import io.netty.channel.ChannelInitializer;
import io.netty.channel.ChannelOption;
import io.netty.channel.EventLoopGroup;
import io.netty.channel.nio.NioEventLoopGroup;
import io.netty.channel.socket.SocketChannel;
import io.netty.channel.socket.nio.NioServerSocketChannel;

public class StartServer {

    private int port;

    public StartServer(int port) {
        this.port = port;
    }

    public void run() throws Exception {
        // Acceptor:threads default is availableProcessors * 2
        EventLoopGroup bossGroup = new NioEventLoopGroup(2);
        // Handler
        EventLoopGroup workerGroup = new NioEventLoopGroup(4);
        try {
            ServerBootstrap server = new ServerBootstrap();
            ChannelHandler handler = new ChannelInitializer<SocketChannel>(){
                @Override
                protected void initChannel(SocketChannel ch) throws Exception {
                    ch.pipeline().addLast(new ConnectionHandler());
                }
            };
            server.group(bossGroup, workerGroup)
                    .channel(NioServerSocketChannel.class)
                    .childHandler(handler)
                    .option(ChannelOption.SO_BACKLOG, 128)
                    .childOption(ChannelOption.SO_KEEPALIVE, true);
            // Start the client
            ChannelFuture future = server.bind(port).sync();

            System.out.println("IM Server start");

            // Wait until the connection is closed
            future.channel().closeFuture().sync();
        } finally {
            workerGroup.shutdownGracefully();
            bossGroup.shutdownGracefully();
        }
    }

    public static void main(String[] args) throws Exception {
        new StartServer(8088).run();

    }
}
EOF

In [ ]:
mkdir -p src/main/java/name/jhub/im/handler

In [ ]:
cat << EOF > src/main/java/name/jhub/im/handler/ConnectionHandler.java
package name.jhub.im.handler;

import java.net.SocketAddress;
import java.nio.charset.Charset;
import java.util.Iterator;

import name.jhub.im.session.LocalChannelManger;

import io.netty.buffer.ByteBuf;
import io.netty.buffer.Unpooled;
import io.netty.buffer.UnpooledByteBufAllocator;
import io.netty.channel.ChannelHandlerContext;
import io.netty.channel.ChannelInboundHandlerAdapter;
import io.netty.util.CharsetUtil;

public class ConnectionHandler extends ChannelInboundHandlerAdapter {

    @Override
    public void channelRegistered(ChannelHandlerContext ctx) throws Exception {
        //SocketAddress address = ctx.channel().remoteAddress();
        //LocalChannelManger.getInstance().addContext(address.toString(), ctx);
        super.channelRegistered(ctx);
    }

    @Override
    public void channelUnregistered(ChannelHandlerContext ctx) throws Exception {
        LocalChannelManger.getInstance().removeContext(ctx);
        syncRoster();
        SocketAddress address = ctx.channel().remoteAddress();
        System.out.println(address.toString() + "channelUnregistered");
        int count = LocalChannelManger.getInstance().staticClients();
        System.out.println("current clients : " + count);
    }

    @Override
    public void channelActive(ChannelHandlerContext ctx) throws Exception {
        System.out.println("channelActive");
    }

    @Override
    public void channelRead(ChannelHandlerContext ctx, Object msg)
            throws Exception {
        ByteBuf in = (ByteBuf) msg;
        String message = in.toString(Charset.forName("UTF-8"));
        // Flash沙箱处理
        String xml = "<?xml version=\"1.0\"?><cross-domain-policy><site-control permitted-cross-domain-policies=\"all\"/><allow-access-from domain=\"*\" to-ports=\"*\"/></cross-domain-policy>\0";
        if(message.trim().equals("<policy-file-request/>")){
            ctx.writeAndFlush(Unpooled.copiedBuffer(xml,CharsetUtil.UTF_8));
        }
        if(message.startsWith("AUTH:")){
            String name = (message.split(":"))[1];
            LocalChannelManger.getInstance().addContext(name, ctx);
            int count = LocalChannelManger.getInstance().staticClients();
            System.out.println("current clients : " + count);
            syncRoster();
        } else if (message.startsWith("MSG:")){
            String content = message.substring(4);
            String[] temp = content.split("#");
            String to = temp[0];
            String body = "";
            for(int i=1;i<temp.length;i++){
                if(i > 1){
                    body += "#";
                }
                body += temp[i];
            }
            if(LocalChannelManger.getInstance().isAvailable(to)){
                LocalChannelManger.getInstance().getContext(to).writeAndFlush(Unpooled.copiedBuffer(body,CharsetUtil.UTF_8));
            }
        } else if (message.startsWith("QUIT:")){
            String name = (message.split(":"))[1];
            LocalChannelManger.getInstance().removeContext(name);
            int count = LocalChannelManger.getInstance().staticClients();
            System.out.println("current clients : " + count);
            syncRoster();
        }
        System.out.println(message);

    }

    @Override
    public void exceptionCaught(ChannelHandlerContext ctx, Throwable cause)
            throws Exception {
        //ctx.close();
        //System.out.println("server closed!");
    }

    // update all clients roster
    private void syncRoster(){
        String respone = "ROSTER:";
        for(String s : LocalChannelManger.getInstance().getAll()){
            respone += s + ",";
        }
        Iterator<ChannelHandlerContext> it = LocalChannelManger.getInstance().getAllClient().iterator();
        while(it.hasNext()){
            it.next().writeAndFlush(Unpooled.copiedBuffer(respone,CharsetUtil.UTF_8));
        }
    }

}
EOF

In [ ]:
mkdir -p src/main/java/name/jhub/im/session

In [ ]:
cat << EOF > src/main/java/name/jhub/im/session/LocalChannelManger.java
package name.jhub.im.session;

import io.netty.channel.ChannelHandlerContext;

import java.util.Collection;
import java.util.Map;
import java.util.Set;
import java.util.concurrent.ConcurrentHashMap;

public class LocalChannelManger {
    // 存储用户名与连接上下文对象的映射
    final private Map<String, ChannelHandlerContext> sessions = new ConcurrentHashMap<String, ChannelHandlerContext>();
    // 存储连接上下文与用户名的映射
    final private Map<String, String> relations = new ConcurrentHashMap<String, String>();

    private static LocalChannelManger instance = new LocalChannelManger();

    public static LocalChannelManger getInstance(){
        return instance;
    }

    // 增加用户与连接的上下文映射
    public void addContext(String name, ChannelHandlerContext ctx){
        synchronized (sessions) {
            sessions.put(name, ctx);
            relations.put(ctx.toString(), name);
        }
    }

    // 获取指定用户的连接上下文
    public ChannelHandlerContext getContext(String name){
        return sessions.get(name);
    }

    // 根据用户名删除session
    public void removeContext(String name){
        sessions.remove(name);
    }

    // 判断指定的用户名当前是否在线
    public boolean isAvailable(String name){
        return sessions.containsKey(name) && (sessions.get(name) != null);
    }

    // 获取所有的用户名
    public synchronized Set<String> getAll(){
        return sessions.keySet();
    }

    // 获取所有连接的上下文对象
    public synchronized Collection<ChannelHandlerContext> getAllClient(){
        return sessions.values();
    }

    // 根据上下文删除用户session
    public void removeContext(ChannelHandlerContext ctx){
        String name = relations.get(ctx.toString());
        if(name != null){
            sessions.remove(name);
            relations.remove(ctx.toString());
        }
    }

    // 统计当前在线人数
    public int staticClients(){
        return relations.size();
    }

}
EOF

### Build

#### Add Netty to pom.xml

``` bash
cat pom.xml
```

``` xml
...
    <!-- https://mvnrepository.com/artifact/io.netty/netty-all -->
    <dependency>
      <groupId>io.netty</groupId>
      <artifactId>netty-all</artifactId>
      <version>4.1.70.Final</version>
      <scope>compile</scope>
    </dependency>
...
```

In [ ]:
cat << EOF > pom.xml
<?xml version="1.0" encoding="UTF-8"?>

<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <groupId>name.jhub.im</groupId>
  <artifactId>instant-messaging</artifactId>
  <version>1.0-SNAPSHOT</version>

  <name>instant-messaging</name>
  <url>https://seii-saintway.github.io/</url>

  <properties>
    <project.build.sourceEncoding>UTF-8</project.build.sourceEncoding>
    <java.version>11</java.version>
  </properties>

  <dependencies>
    <!-- https://mvnrepository.com/artifact/io.netty/netty-all -->
    <dependency>
      <groupId>io.netty</groupId>
      <artifactId>netty-all</artifactId>
      <version>4.1.70.Final</version>
      <scope>compile</scope>
    </dependency>
    <dependency>
      <groupId>junit</groupId>
      <artifactId>junit</artifactId>
      <version>4.11</version>
      <scope>test</scope>
    </dependency>
  </dependencies>

  <build>
    <plugins>
      <plugin>
        <artifactId>maven-clean-plugin</artifactId>
        <version>3.1.0</version>
      </plugin>
      <plugin>
        <artifactId>maven-resources-plugin</artifactId>
        <version>3.0.2</version>
      </plugin>
      <plugin>
        <artifactId>maven-compiler-plugin</artifactId>
        <version>3.8.0</version>
        <configuration>
          <excludes>
            <exclude>**/.ipynb_checkpoints/*.java</exclude>
          </excludes>
        </configuration>
      </plugin>
      <plugin>
        <artifactId>maven-surefire-plugin</artifactId>
        <version>2.22.1</version>
      </plugin>
      <plugin>
        <artifactId>maven-jar-plugin</artifactId>
        <version>3.0.2</version>
      </plugin>
      <plugin>
        <groupId>org.apache.maven.plugins</groupId>
        <artifactId>maven-assembly-plugin</artifactId>
        <version>3.3.0</version>
        <configuration>
          <archive>
            <manifest>
              <mainClass>name.jhub.im.server.StartServer</mainClass>
            </manifest>
          </archive>
          <descriptorRefs>
            <descriptorRef>jar-with-dependencies</descriptorRef>
          </descriptorRefs>
        </configuration>
        <executions>
          <execution>
            <phase>package</phase>
            <goals>
              <goal>single</goal>
            </goals>
          </execution>
        </executions>
      </plugin>
    </plugins>
  </build>
</project>
EOF

In [ ]:
mvn clean

[INFO] Scanning for projects...
[INFO] 
[INFO] -------------------< name.jhub.im:instant-messaging >-------------------
[INFO] Building instant-messaging 1.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:3.1.0:clean (default-clean) @ instant-messaging ---
[INFO] Deleting /home/jovyan/jhub/!-instant-messaging/back-end/target
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  1.113 s
[INFO] Finished at: 2021-11-30T08:38:45Z
[INFO] ------------------------------------------------------------------------


In [ ]:
mvn package

[INFO] Scanning for projects...
[INFO] 
[INFO] -------------------< name.jhub.im:instant-messaging >-------------------
[INFO] Building instant-messaging 1.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:3.0.2:resources (default-resources) @ instant-messaging ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /home/jovyan/jhub/!-instant-messaging/back-end/src/main/resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.0:compile (default-compile) @ instant-messaging ---
[INFO] Changes detected - recompiling the module!
[INFO] Compiling 3 source files to /home/jovyan/jhub/!-instant-messaging/back-end/target/classes
[INFO] 
[INFO] --- maven-resources-plugin:3.0.2:testResources (default-testResources) @ instant-messaging ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /home/jovyan/jhub/!-instant-m

In [ ]:
# jar -tf target/instant-messaging-1.0-SNAPSHOT-jar-with-dependencies.jar

### Run

In [ ]:
java -jar target/instant-messaging-1.0-SNAPSHOT-jar-with-dependencies.jar

IM Server start


### Test